In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import csv
import re

# Initialize the Chrome WebDriver
driver = webdriver.Chrome()

# Define the URL of the page containing the links
url = 'https://matadornetwork.com/destinations/'  # Replace with the actual URL

# Open CSV file for writing
csv_file = open('West.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['City', 'Header', 'Title', 'Description', 'Image URL'])

try:
    try:
        # Open the page
        driver.get(url)
    except TimeoutException:
        print("Timeout occurred while loading the main page. Exiting...")
        driver.quit()

    while True:
        # Find all city links
        city_links = driver.find_elements(By.CSS_SELECTOR, 'a.destination-list__item')

        # Check if there are no links left to click
        if not city_links:
            break

        for index in range(len(city_links)):
            try:
                # Re-find all city links
                city_links = driver.find_elements(By.CSS_SELECTOR, 'a.destination-list__item')
                
                # Get the link for the current index
                link = city_links[index]
                city_name = link.text
                city_url = link.get_attribute('href')

                print(f"Clicking on city: {city_name}")
                link.click()  # Click the city link

                # Wait for the page to load
                time.sleep(3)  # Adjust sleep time as needed

                # Find all article links on the page
                article_links = driver.find_elements(By.CSS_SELECTOR, 'a.article-link-card__content')

                # Iterate over the article links and click each one
                for article_link in article_links:
                    header = article_link.find_element(By.TAG_NAME, 'h2').text

                    # Check if the header should be excluded (contains "Day One", "Day Two", etc.)
                    if re.search(r'\bDAY\s+(ONE|TWO|THREE|FOUR|FIVE|SIX|SEVEN)\b', header, re.IGNORECASE):
                        print(f"Skipping Itinerary Header: {header}")
                        continue  # Skip this header

                    print(f"Header: {header}")

                    # Open the link in a new tab
                    article_link.send_keys(Keys.CONTROL + Keys.RETURN)  # Ctrl+Click to open in a new tab
                    driver.switch_to.window(driver.window_handles[-1])  # Switch to the new tab

                    # Wait for the new tab to load
                    time.sleep(3)  # Adjust sleep time as needed

                    # Get the page source and parse it with BeautifulSoup
                    soup = BeautifulSoup(driver.page_source, 'html.parser')

                    # Find all the relevant divs containing image information
                    image_divs = soup.find_all('div', class_='wp-caption')

                    for div in image_divs:
                        # Extract the image URL
                        img_tag = div.find('img')
                        img_url = img_tag['src']
                        
                        # Filter only JPG images
                        if img_url.endswith('.jpg'):
                            # Extract the title (heading of the section)
                            title = div.find_previous('h2').text.strip()
                            
                            # Extract the caption tag
                            caption_tag = div.find('p', class_='wp-caption-text')
                            if caption_tag:
                                # Find the next <p> tag after the photo tag
                                next_p = div.find_next_sibling('p')
                                description = next_p.get_text(strip=True) if next_p else 'No description available'
                                
                                print(f"Title: {title}")
                                print(f"Image URL: {img_url}")
                                print(f"Caption Tag: {caption_tag.text.strip()}")
                                print(f"Description: {description}")
                                print()

                                # Write the data to the CSV file
                                csv_writer.writerow([city_name, header, title, description, img_url])

                    # Close the tab after processing
                    driver.close()

                    # Switch back to the original tab
                    driver.switch_to.window(driver.window_handles[0])

                # Go back to the original page with the list of cities
                driver.back()

                # Wait for the list page to load again
                time.sleep(3)  # Adjust sleep time as needed

            except StaleElementReferenceException:
                print("Stale element reference encountered. Re-finding elements...")
                # Handle the exception by re-finding the city links
                city_links = driver.find_elements(By.CSS_SELECTOR, 'a.destination-list__item')

            except TimeoutException:
                print("Timeout occurred while loading the page. Continuing with the next link...")
                driver.back()  # Return to the previous page and continue

finally:
    # Close the CSV file and WebDriver
    csv_file.close()
    driver.quit()


The chromedriver version (126.0.6478.126) detected in PATH at /usr/local/bin/chromedriver might not be compatible with the detected chrome version (127.0.6533.119); currently, chromedriver 127.0.6533.119 is recommended for chrome 127.*, so it is advised to delete the driver in PATH and retry


Clicking on city: Abu Dhabi
Clicking on city: Amsterdam
Header: NEIGHBORHOODS
Title: Itineraries
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/11/Dam-Square-in-Amsterdam.jpg
Caption Tag: Photo: Olena Z/Shutterstock
Description: The historic center— Amsterdam’s historic center radiates from Dam Square, where a dam kept the city from flooding in the 13th century. Now a stage for celebrations, street entertainment, and social activism, it’s also where Atlas hoists the universe atop theRoyal Palace, symbolizing the city’s 17th-century world dominance. Once occupied by Napoleon, it’s the only palace in theNetherlandsthat’s both open to the public and still used for royal receptions.De Nieuwe Kerk(The New Church), where King Willem-Alexander married Maxima in 2002 and was crowned the Netherlands’ first king in a century following his mother’s abdication in 2013, sits on one side of the square.

Title: Itineraries
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/1

Title: Itineraries
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/11/Van-Gogh-Museum.jpg
Caption Tag: Photo: Alicia G. Monedero/Shutterstock
Description: Van Gogh Museum— Set on Museumplein, the world’s largest collection of paintings by Dutch painter Vincent van Gogh showcases “Starry Night,” “Sunflowers,” “The Potato Eaters,” “Almond Blossom,” “The Bedroom,” and other renowned masterpieces. Visitors can admire the painter’s singular style through still lives, portraits, self-portraits, and landscapes. Book a private tour, attend a walk-in workshop, or visit on the last Friday evening of the month when DJs and other creative talent add to the viewing experience.

Title: Itineraries
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/11/Het-Concertgebouw-Amsterdam.jpg
Caption Tag: Photo: Het Concertgebouw Amsterdam/Facebook
Description: Concertgebouw— Exceptional acoustics and quality orchestral, operatic, jazz, and world music programs make Amsterdam’s concert

Title: Itineraries
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/11/Crowd-of-people-dressed-in-orange-celebrate-Kings-Day-in-Amsterdam.jpg
Caption Tag: Photo: lornet/Shutterstock
Description: King’s Day— Little can prepare you for the orange-tinged madness of King’s Day, orKoningsdag, the national holiday that gives the Dutch an excuse to party hard for over 24 hours. In Amsterdam alone, over a million revelers spill onto streets and canals, ostensibly to celebrate King Willem-Alexander’s birthday on April 27 (if his birthday lands on a Sunday, the holiday is celebrated on April 26). The fun begins the night before on King’s Night (Koningsnacht) when bars and public squares overflow with people. The next morning, the city turns into a giant flea market,or vrijmarkt, which is the only day anyone can sell anything on the street tax-free. As the afternoon winds down, the party ramps up with live music, street vendors, pop-up bars, and dancing in the streets.

Title: Itiner

Header: BARS AND NIGHTLIFE
Title: Central Station Waterfront
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/11/Hannekes-Boom.jpg
Caption Tag: Photo: Hannekes Boom/Facebook
Description: Hannekes Boom$$ — A relaxed gastro-bar on the water that’s furnished with recycled furniture. Especially popular in the summer when it’s easiest to take full advantage of the sunshine, you can moor your boat in what was once a 17th-century guardhouse to get to Hannekes Boom. Once there, grab a drink and dangle your toes from a waterfront terrace overlooking NEMO and the Amsterdam skyline.

Title: Dam Square/Centrum
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/11/Club-NL.jpg
Caption Tag: Photo: Club NL/Facebook
Description: Club NL$$ — The interior of Club NL is dominated by red velvet, dark woods, and nude drawings, and it’s drawn in celebrities like Mick Jagger and Naomi Campbell. The focus here is on house and techno music, attracting top DJs and people to the dance floo

...

Title: Hotels
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/10/Morris-House-Hotel.jpg
Caption Tag: Photo: Morris House Hotel/Facebook
Description: Morris House— At first glance, this colonial-style mansion looks like just another one of Philadelphia’s many historic buildings. And while Morris House is a National Historic Landmark, it’s also an old-fashioned hotel that doesn’t skip on modern comforts. Every morning, the cozy Revolutionary-era lounge serves fresh tea, coffee, and cookies to guests. Morris House also has a scenic garden view of the surrounding Washington Square Park, perfect for anyone who needs a little nature during their city vacation.

Title: Hotels
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/10/Penns-View-Hotel-Philly.jpg
Caption Tag: Photo: Penn’s View Hotel
Description: Penn’s View— This boutique hotel is located in the heart of Old City, within walking distance of the city’s most famous landmarks. Penn’s View uses a colonial aesth

Title: Itineraries
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/10/Blue-Cross-Broad-Street-Run.jpg
Caption Tag: Photo: Blue Cross Broad Street Run/Facebook
Description: Broad Street Run— The Broad Street Run, going strong since 1980, is the largest 10-mile race in the US. Every year, at the beginning of May, at least 35,000 participants flock to Broad Street from every corner of the world. It takes you throughout the city, starting in North Philadelphia and passing by Temple University, City Hall, Avenue of the Arts, and the South Philadelphia Sports Complex. Don’t worry if you aren’t a runner — there are refreshments and entertainment all along Broad Street. Don’t miss the finish line party at South Philly’s Navy Yard.

Title: Itineraries
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/10/Wawa-Welcome-America.jpg
Caption Tag: Photo: Wawa Welcome America/Facebook
Description: Wawa Welcome America— Philadelphia is one of the best places in all of the US to

Title: South Philadelphia
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/10/The-Dolphin-Tavern.jpg
Caption Tag: Photo: The Dolphin Tavern/Facebook
Description: Dolphin Tavern$ — This South Philly dance club knows how to throw a party without busting your bank account. The club knits together indie style, light up dance floors, and burlesque dancers. There’s always something going on, whether it’s Disco Italiana night or throwback themes from the 1980s to the early 2000s.

Title: Northern Liberties
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/10/Standard-Tap.jpg
Caption Tag: Photo: Standard Tap/Facebook
Description: Standard Tap$$ — Located in a large white farmhouse, Standard Tap is one of the city’s oldest gastropubs and has a friendly, casual atmosphere. Standard Tap gives you a truly authentic Philly experience, and it takes pride in only using regional produce on the food menu and beers from local microbreweries at the bar. As an added bonus, it stri

Title: Itineraries
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/10/Bucks-County-Playhouse-theater-in-New-Hope-Pennsylvania.jpg
Caption Tag: Photo: EQRoy/Shutterstock
Description: New Hope, Pennsylvania— New Hope’s known as a cool suburb (in fact, it was named one ofMatador Network’scoolest towns in Americain 2018), and it definitely lives up to that reputation. This Delaware River town, with gorgeous, old architecture and dogwood trees, might be quieter than the city, but it’s definitely not boring. New Hope has several niche small businesses, specializing in anything from lingerie to Irish imports to magic. You can grab a meal at theBowman’s Tavernor the vegan-friendlyLambertville Station Restaurant and Inn. There are also art galleries, rumors of a haunted inn (theLogan Inn), and an impossible-to-miss big red barn that hosts local theater. Interestingly enough, two of its biggest communities are bikers and drag queens, and they get along swimmingly. New Hope is only 

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.119)
Stacktrace:
#0 0x55e074d39c7a <unknown>
#1 0x55e074a1ce2c <unknown>
#2 0x55e0749f20d4 <unknown>
#3 0x55e074a94a8d <unknown>
#4 0x55e074aaaca6 <unknown>
#5 0x55e074a8c363 <unknown>
#6 0x55e074a5c247 <unknown>
#7 0x55e074a5cb9e <unknown>
#8 0x55e074d0024b <unknown>
#9 0x55e074d042f1 <unknown>
#10 0x55e074cebafe <unknown>
#11 0x55e074d04e52 <unknown>
#12 0x55e074cd079f <unknown>
#13 0x55e074d29638 <unknown>
#14 0x55e074d29810 <unknown>
#15 0x55e074d38dac <unknown>
#16 0x73ff85694ac3 <unknown>


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import csv
import re

# Initialize the Chrome WebDriver
driver = webdriver.Chrome()

# Define the URL of the page containing the links
url = 'https://matadornetwork.com/destinations/'  # Replace with the actual URL

# Open CSV file for writing
csv_file = open('Night.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['City', 'Header', 'Title', 'Description', 'Image URL', 'Photo Tag'])

try:
    try:
        # Open the page
        driver.get(url)
    except TimeoutException:
        print("Timeout occurred while loading the main page. Exiting...")
        driver.quit()

    while True:
        # Find all city links
        city_links = driver.find_elements(By.CSS_SELECTOR, 'a.destination-list__item')

        # Check if there are no links left to click
        if not city_links:
            break

        for index in range(len(city_links)):
            try:
                # Re-find all city links
                city_links = driver.find_elements(By.CSS_SELECTOR, 'a.destination-list__item')
                
                # Get the link for the current index
                link = city_links[index]
                city_name = link.text
                city_url = link.get_attribute('href')

                print(f"Clicking on city: {city_name}")
                link.click()  # Click the city link

                # Wait for the page to load
                time.sleep(3)  # Adjust sleep time as needed

                # Find all article links on the page
                article_links = driver.find_elements(By.CSS_SELECTOR, 'a.article-link-card__content')

                # Iterate over the article links and click each one
                for article_link in article_links:
                    header = article_link.find_element(By.TAG_NAME, 'h2').text

                    # Check if the header should be excluded (contains "Day One", "Day Two", etc.)
                    if re.search(r'\bDAY\s+(ONE|TWO|THREE|FOUR|FIVE|SIX|SEVEN)\b', header, re.IGNORECASE):
                        print(f"Skipping Itinerary Header: {header}")
                        continue  # Skip this header

                    print(f"Header: {header}")

                    # Open the link in a new tab
                    article_link.send_keys(Keys.CONTROL + Keys.RETURN)  # Ctrl+Click to open in a new tab
                    driver.switch_to.window(driver.window_handles[-1])  # Switch to the new tab

                    # Wait for the new tab to load
                    time.sleep(3)  # Adjust sleep time as needed

                    # Get the page source and parse it with BeautifulSoup
                    soup = BeautifulSoup(driver.page_source, 'html.parser')

                    # Find all the relevant divs containing image information
                    image_divs = soup.find_all('div', class_='wp-caption')

                    for div in image_divs:
                        # Extract the image URL
                        img_tag = div.find('img')
                        img_url = img_tag['src'] if img_tag else 'No image available'
                        
                        # Filter only JPG images
                        if img_url.endswith('.jpg'):
                            # Extract the title (heading of the section)
                            title_tag = div.find_previous('h2')
                            title = title_tag.text.strip() if title_tag else 'No title available'
                            
                            # Extract the caption tag
                            caption_tag = div.find('p', class_='wp-caption-text')
                            caption = caption_tag.text.strip() if caption_tag else 'No caption available'
                            
                            # Extract the photo tag (e.g., "Photo: Harry Beugelink/Shutterstock")
                            photo_tag = caption_tag.get_text(strip=True) if caption_tag else 'No photo tag available'
                            
                            # Extract the description (next <p> tag)
                            next_p = div.find_next_sibling('p')
                            description = next_p.get_text(strip=True) if next_p else 'No description available'
                            
                            print(f"Title: {title}")
                            print(f"Image URL: {img_url}")
                            print(f"Caption: {caption}")
                            print(f"Photo Tag: {photo_tag}")
                            print(f"Description: {description}")
                            print()

                            # Write the data to the CSV file
                            csv_writer.writerow([city_name, header, title, description, img_url, photo_tag])

                    # Close the tab after processing
                    driver.close()

                    # Switch back to the original tab
                    driver.switch_to.window(driver.window_handles[0])

                # Go back to the original page with the list of cities
                driver.back()

                # Wait for the list page to load again
                time.sleep(3)  # Adjust sleep time as needed

            except StaleElementReferenceException:
                print("Stale element reference encountered. Re-finding elements...")
                # Handle the exception by re-finding the city links
                city_links = driver.find_elements(By.CSS_SELECTOR, 'a.destination-list__item')

            except TimeoutException:
                print("Timeout occurred while loading the page. Continuing with the next link...")
                driver.back()  # Return to the previous page and continue

finally:
    # Close the CSV file and WebDriver
    csv_file.close()
    driver.quit()


The chromedriver version (126.0.6478.126) detected in PATH at /usr/local/bin/chromedriver might not be compatible with the detected chrome version (127.0.6533.119); currently, chromedriver 127.0.6533.119 is recommended for chrome 127.*, so it is advised to delete the driver in PATH and retry


Clicking on city: Abu Dhabi
Clicking on city: Amsterdam
Header: NEIGHBORHOODS
Title: Itineraries
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/11/Rembrandt-van-Rijn-monument.jpg
Caption: Photo: kamienczanka/Shutterstock
Photo Tag: Photo:kamienczanka/Shutterstock
Description: De Plantage— Far greener and less touristy than Amsterdam’s historic center, De Plantage has wide, leafy boulevards that contrast with the Canal Belt’s narrow, cobblestoned streets. The neighborhood is home to the Hortus botanical gardens and the city’s Jewish Quarter, where Rembrandt lived at the height of his fame. Today the famous painter’s house is amuseumreplete with prints, sculptures, and paintings that reconstruct his life in the Dutch Golden Age.

Title: Itineraries
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/11/NDSM-Wharf-Amsterdam-1.jpg
Caption: Photo: fokke baarssen/Shutterstock
Photo Tag: Photo:fokke baarssen/Shutterstock
Description: Noord-Amsterdam— Free ferries on t

Title: Itineraries
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/11/Van-Gogh-Museum.jpg
Caption: Photo: Alicia G. Monedero/Shutterstock
Photo Tag: Photo:Alicia G. Monedero/Shutterstock
Description: Van Gogh Museum— Set on Museumplein, the world’s largest collection of paintings by Dutch painter Vincent van Gogh showcases “Starry Night,” “Sunflowers,” “The Potato Eaters,” “Almond Blossom,” “The Bedroom,” and other renowned masterpieces. Visitors can admire the painter’s singular style through still lives, portraits, self-portraits, and landscapes. Book a private tour, attend a walk-in workshop, or visit on the last Friday evening of the month when DJs and other creative talent add to the viewing experience.

Title: Itineraries
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/11/Het-Concertgebouw-Amsterdam.jpg
Caption: Photo: Het Concertgebouw Amsterdam/Facebook
Photo Tag: Photo:Het Concertgebouw Amsterdam/Facebook
Description: Concertgebouw— Exceptional acou

Title: Itineraries
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/11/Crowd-of-people-dressed-in-orange-celebrate-Kings-Day-in-Amsterdam.jpg
Caption: Photo: lornet/Shutterstock
Photo Tag: Photo:lornet/Shutterstock
Description: King’s Day— Little can prepare you for the orange-tinged madness of King’s Day, orKoningsdag, the national holiday that gives the Dutch an excuse to party hard for over 24 hours. In Amsterdam alone, over a million revelers spill onto streets and canals, ostensibly to celebrate King Willem-Alexander’s birthday on April 27 (if his birthday lands on a Sunday, the holiday is celebrated on April 26). The fun begins the night before on King’s Night (Koningsnacht) when bars and public squares overflow with people. The next morning, the city turns into a giant flea market,or vrijmarkt, which is the only day anyone can sell anything on the street tax-free. As the afternoon winds down, the party ramps up with live music, street vendors, pop-up bars, and danci

Title: Central Station Waterfront
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/11/Hannekes-Boom.jpg
Caption: Photo: Hannekes Boom/Facebook
Photo Tag: Photo:Hannekes Boom/Facebook
Description: Hannekes Boom$$ — A relaxed gastro-bar on the water that’s furnished with recycled furniture. Especially popular in the summer when it’s easiest to take full advantage of the sunshine, you can moor your boat in what was once a 17th-century guardhouse to get to Hannekes Boom. Once there, grab a drink and dangle your toes from a waterfront terrace overlooking NEMO and the Amsterdam skyline.

Title: Dam Square/Centrum
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/11/Club-NL.jpg
Caption: Photo: Club NL/Facebook
Photo Tag: Photo:Club NL/Facebook
Description: Club NL$$ — The interior of Club NL is dominated by red velvet, dark woods, and nude drawings, and it’s drawn in celebrities like Mick Jagger and Naomi Campbell. The focus here is on house and techno music, attracti

...

Title: Hotels
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/10/Morris-House-Hotel.jpg
Caption: Photo: Morris House Hotel/Facebook
Photo Tag: Photo:Morris House Hotel/Facebook
Description: Morris House— At first glance, this colonial-style mansion looks like just another one of Philadelphia’s many historic buildings. And while Morris House is a National Historic Landmark, it’s also an old-fashioned hotel that doesn’t skip on modern comforts. Every morning, the cozy Revolutionary-era lounge serves fresh tea, coffee, and cookies to guests. Morris House also has a scenic garden view of the surrounding Washington Square Park, perfect for anyone who needs a little nature during their city vacation.

Title: Hotels
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/10/Penns-View-Hotel-Philly.jpg
Caption: Photo: Penn’s View Hotel
Photo Tag: Photo:Penn’s View Hotel
Description: Penn’s View— This boutique hotel is located in the heart of Old City, within walking distanc

Title: Itineraries
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/10/Wawa-Welcome-America.jpg
Caption: Photo: Wawa Welcome America/Facebook
Photo Tag: Photo:Wawa Welcome America/Facebook
Description: Wawa Welcome America— Philadelphia is one of the best places in all of the US to celebrate the Fourth of July — as it should be, since the Declaration of Independence was signed here. Each year, Wawa (a convenience store/hoagie shop/gas station that is super popular here) throws a week-long party across the city. There are dozens of events each day, like beer gardens, barbecues, film screenings, and block parties. It all culminates in a massive concert and fireworks display on Benjamin Franklin Parkway, in front of the art museum.

Title: Itineraries
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/10/FringeArts.jpg
Caption: Photo: FringeArts/Facebook
Photo Tag: Photo:FringeArts/Facebook
Description: Fringe Festival— September is one of the best times for art lo

Title: Center City
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/10/Tattooed-Mom-1.jpg
Caption: Photo: Tattooed Mom/Facebook
Photo Tag: Photo:Tattooed Mom/Facebook
Description: Tattooed Mom$ — Tattooed Mom embraces South Street’s special style. The bar focus is on beer and custom, often off-beat, cocktails like the Xmas Punk (a holiday cocktail with applejack, Port, and ginger liqueur) and Pickletini (yes, that’s a pickle-flavored Martini). Vegans love it for its vegetarian menu with options like veggie corn dogs. If that’s not enough to make you love this spot, there are two happy hours every day (one from 5:00-7:00 PM and a later one from 10:00-11:00 PM) as well as alternative events like SLAYHer, which is a heavy metal drag show.

Title: Center City
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/10/Sampan-Restaurant-Philadelphia.jpg
Caption: Photo: Sampan Restaurant Philadelphia/Facebook
Photo Tag: Photo:Sampan Restaurant Philadelphia/Facebook
Descript

Title: Itineraries
Image URL: https://d36tnp772eyphs.cloudfront.net/blogs/1/2019/10/Bucks-County-Playhouse-theater-in-New-Hope-Pennsylvania.jpg
Caption: Photo: EQRoy/Shutterstock
Photo Tag: Photo:EQRoy/Shutterstock
Description: New Hope, Pennsylvania— New Hope’s known as a cool suburb (in fact, it was named one ofMatador Network’scoolest towns in Americain 2018), and it definitely lives up to that reputation. This Delaware River town, with gorgeous, old architecture and dogwood trees, might be quieter than the city, but it’s definitely not boring. New Hope has several niche small businesses, specializing in anything from lingerie to Irish imports to magic. You can grab a meal at theBowman’s Tavernor the vegan-friendlyLambertville Station Restaurant and Inn. There are also art galleries, rumors of a haunted inn (theLogan Inn), and an impossible-to-miss big red barn that hosts local theater. Interestingly enough, two of its biggest communities are bikers and drag queens, and they get alo

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.119)
Stacktrace:
#0 0x606962257c7a <unknown>
#1 0x606961f3ae2c <unknown>
#2 0x606961f100d4 <unknown>
#3 0x606961fb2a8d <unknown>
#4 0x606961fc8ca6 <unknown>
#5 0x606961faa363 <unknown>
#6 0x606961f7a247 <unknown>
#7 0x606961f7ab9e <unknown>
#8 0x60696221e24b <unknown>
#9 0x6069622222f1 <unknown>
#10 0x606962209afe <unknown>
#11 0x606962222e52 <unknown>
#12 0x6069621ee79f <unknown>
#13 0x606962247638 <unknown>
#14 0x606962247810 <unknown>
#15 0x606962256dac <unknown>
#16 0x7eedfd494ac3 <unknown>


In [4]:
import requests  
import time  
import csv  

# Replace with your ParseHub API key and project token  
API_KEY = 'tMon_iunYRBt'  
PROJECT_TOKEN = 'tZ9wfVBqBbOS'  

# Start the ParseHub run  
def start_parsehub_run():  
    url = 'https://www.parsehub.com/api/v2/projects/{}/run'.format(PROJECT_TOKEN)  
    response = requests.post(url, data={'api_key': API_KEY})  
    return response.json()  

# Get the results of the run  
def get_parsehub_results(run_token):  
    url = 'https://www.parsehub.com/api/v2/runs/{}/data'.format(run_token)  
    response = requests.get(url, params={'api_key': API_KEY})  
    return response.json()  

# Save results to CSV  
def save_to_csv(data, filename='scraped_data.csv'):  
    # Open a file for writing  
    with open(filename, mode='w', newline='', encoding='utf-8') as file:  
        writer = csv.writer(file)  
        
        # Write header  
        header = data[0].keys()  # Get the keys from the first dictionary as header  
        writer.writerow(header)  
        
        # Write data rows  
        for row in data:  
            writer.writerow(row.values())  

# Main function to execute the scraping  
def main():  
    # Start the scraping run  
    run_response = start_parsehub_run()  
    run_token = run_response.get('run_token')  
    
    print("Run started. Run token:", run_token)  

    # Wait for the run to complete  
    while True:  
        time.sleep(10)  # Wait for 10 seconds before checking the status  
        run_status_url = 'https://www.parsehub.com/api/v2/runs/{}'.format(run_token)  
        status_response = requests.get(run_status_url, params={'api_key': API_KEY}).json()  
        
        if status_response['status'] == 'finished':  
            print("Run finished.")  
            break  
        elif status_response['status'] == 'error':  
            print("Error in run:", status_response['error'])  
            return  

    # Get the results  
    results = get_parsehub_results(run_token)  
    print("Scraped Data:", results)  

    # Save results to CSV  
    save_to_csv(results)  

if _name_ == "_main_":  
    main()

NameError: name '_name_' is not defined

In [5]:
import pandas as pd

# Define file paths
input_file = '/home/erk/Desktop/Night.csv'
output_file = 'home/erk/Desktop/Updated_Night.csv'

# Read the CSV file
df = pd.read_csv(input_file)

# Create new columns for the title and description
df['Title'] = df['Description'].apply(lambda x: x.split('—')[0].strip())
df['Description'] = df['Description'].apply(lambda x: '—'.join(x.split('—')[1:]).strip() if '—' in x else '')

# Drop the original Description column if needed
df.drop(columns=['Description'], inplace=True)

# Save the updated DataFrame to a new CSV file
df.to_csv(output_file, index=False)

print(f"Updated CSV file saved as {output_file}")


AttributeError: 'float' object has no attribute 'split'

In [13]:
import pandas as pd
import os

# Load the CSV file
file_path = '/home/erk/Desktop/Night.csv'
df = pd.read_csv(file_path)

# Function to separate title from description
def split_title_description(text):
    if pd.isna(text):
        return '', ''  # Handle NaN values
    if '—' in text:
        parts = text.split('—', 1)
        title = parts[0].strip()
        description = parts[1].strip() if len(parts) > 1 else ''
        return title, description
    return text, ''  # Handle cases where '—' is not in the text

# Apply the function to the 'Description' column
df[['Title', 'Description_Split']] = df['Description'].apply(split_title_description).apply(pd.Series)

# Print column names and the first few rows to verify
print("Columns after splitting:", df.columns)
print("First few rows after splitting:")
print(df.head())

# Drop the old 'Description' column and rename 'Description_Split' to 'Description'
df = df.drop(columns=['Description'])
df = df.rename(columns={'Description_Split': 'Description'})

# Print column names to verify
print("Columns after dropping and renaming:", df.columns)

# Reorder columns and ensure all are present
required_columns = ['City', 'Header', 'Title', 'Description', 'Image URL', 'Photo Tag']
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    print(f"Missing columns: {missing_columns}")
else:
    df = df[required_columns]

# Ensure the directory exists
directory = '/home/erk/Desktop'
if not os.path.exists(directory):
    os.makedirs(directory)

# Save the new CSV file
df.to_csv(os.path.join(directory, 'Updated_Night12.csv'), index=False, encoding='utf-8')

print("CSV file has been processed and saved as 'Updated_Night12.csv'.")


Columns after splitting: Index(['City', 'Header', 'Description', 'Image URL', 'Photo Tag', 'Title',
       'Description_Split'],
      dtype='object')
First few rows after splitting:
        City         Header  \
0  Amsterdam  NEIGHBORHOODS   
1  Amsterdam  NEIGHBORHOODS   
2  Amsterdam  WHERE TO STAY   
3  Amsterdam  WHERE TO STAY   
4  Amsterdam  WHERE TO STAY   

                                         Description  \
0  De Plantage— Far greener and less touristy tha...   
1  Noord-Amsterdam— Free ferries on the IJ River ...   
2  The Exchange— Fashion meets interior design in...   
3  Conscious Hotels— Solar panels, desks crafted ...   
4  Hotel not Hotel— Each guest room has a differe...   

                                           Image URL  \
0  https://d36tnp772eyphs.cloudfront.net/blogs/1/...   
1  https://d36tnp772eyphs.cloudfront.net/blogs/1/...   
2  https://d36tnp772eyphs.cloudfront.net/blogs/1/...   
3  https://d36tnp772eyphs.cloudfront.net/blogs/1/...   
4  https://d3